## This notebook walks through the basics of generating synthetic choice data and fitting the model to that data to recover the paramaters that generated it

Fitting these models to data to choice data alone can be computational expensive. We fit the model below using 44 cpus.

`using` is the julia version of python's `import`, i.e. in incorporates the exported functions of a module into the current namespace. `Distributed` is a julia module for performing parallel computing. `addprocs()` adds some workers, which can be called by a main process if there are calls to do things in parallel. `PulseInputDDM` parallelizes the computation of the log-likelhood across trials.

In [1]:
using Distributed
addprocs(8);

### Fitting a choice model.

when `using` is called after workers have been made, i.e. above, then those modules are available on all the workers (i.e. any function called from those modules will be able to be executed on any worker)

In [2]:
@everywhere using PulseInputDDM, Flatten

### Generate some data

Create an instance of the `θchoice` [composite type](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) which contains the 9 parameters of a choice DDM model. 

In [3]:
using PyPlot

In [4]:
θ_syn = θchoice(θz=θz(σ2_i = 1., B = 13., λ = -0.5, σ2_a = 10., σ2_s = 1.0,
    ϕ = 0.4, τ_ϕ = 0.02), bias=0.1, lapse=0.1);

In [5]:
xgen = collect(Flatten.flatten(θ_syn));

Generate 8K trials of the synthetic data using those parameters. change `rng` to get a different set with the same parameters. `dt` specifies the temporal binning of the data. `1e-2` has worked well.

In [6]:
_, data = synthetic_data(;θ=θ_syn, ntrials=8_000, rng=2, dt=1e-2);

`PulseInputDDM` computes the log-likelihood by propogating mass of the latent distribution in time. This is done numerically by dividing the distribution up into little temporal and spatial bins. `n` is the number of spatials bins to use. `n=53` seems to work well.

In [7]:
n = 53

53

Create an instance of the `choiceDDM` composite type which contains the parameters and the data of a choice DDM model. 

In [8]:
model_gen = choiceDDM(θ=θ_syn)


choiceDDM{θchoice{θz{Float64}, Float64}}
  θ: θchoice{θz{Float64}, Float64}
  n: Int64 53
  cross: Bool false


Compute the loglikelihood of the model under the generative parameters

In [9]:
loglikelihood(model_gen, data)

-2969.9823448565967

random initial state

In [10]:
x0 = vcat([0.1, 15., -0.1, 20., 0.5, 0.2, 0.008], [0.,0.01]);

use `Flatten.reconstruct` to place the intial state into the `θchoice()` type, with each variable going into the right place

In [11]:
θ = Flatten.reconstruct(θchoice(), x0)

θchoice{θz{Float64}, Float64}
  θz: θz{Float64}
  bias: Float64 0.0
  lapse: Float64 0.01


In [12]:
model = choiceDDM(θ=θ);

Compute the loglikelihood of the model under the initial parameters

In [13]:
loglikelihood(model, data)

-3473.431290856813

Create an instance of the `choiceoptions` composite type, using `lb`, `ub`, and `fit`. `choiceoptions` specifies which parameters to fit, and the lower and upper bounds of the optimization.

In [14]:
#options = choiceoptions(fit=fit, lb=lb, ub=ub)

In [15]:
options = choiceoptions(lb=vcat([0., 2.,  -5., 0.,   0.,  0., 0.005, -5.0, 0.0]),
    ub = [30., 100., 5., 200., 10., 1.2,  1., 5.0, 1.0], 
    fit = trues(9))

choiceoptions
  fit: Array{Bool}((9,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1]
  lb: Array{Float64}((9,)) [0.0, 2.0, -5.0, 0.0, 0.0, 0.0, 0.005, -5.0, 0.0]
  ub: Array{Float64}((9,)) [30.0, 100.0, 5.0, 200.0, 10.0, 1.2, 1.0, 5.0, 1.0]


Define bounds for each parameter for the optimization. Some of these are strict (i.e. variance > 0) and some are not (B < 100 seems reasonable, but B could of course be larger then 100.)

### Optimize the model, using the options to define details of the optimization

In [16]:
model, output = fit(model, data, options, f_tol=1e-6, iterations = 100, extended_trace=true, show_trace=true)

Fminbox
-------
Initial mu = 0.160987

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.160987

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     3.481363e+03     5.400220e+04
 * Current step size: 1.0
 * time: 0.021502017974853516
 * g(x): [-23.743724932868748, -11.064054277815194, 24.1073365458685, -13.409269237364864, -667.8315427815999, 504.5657745923053, -54002.201879413646, -29.71329184576252, -17891.119702011783]
 * ~inv(H): [1.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.0 -0.0 1.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0; 0.0 0.0 -0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 1.0]
 * x: [0.1, 15.0, -0.1, 20.0, 0.5, 0.2, 0.008, 0.0, 0.01]


Compute the likelihood of the optimized model

In [ ]:
loglikelihood(model, data)

Compute the gradient of the model after optimization, to ensure we are near the minimum of the loglikelihood

In [ ]:
gradient(model, data)

Compute the Hessian of the model, to compute confidence bounds around the ML parameters

In [ ]:
H = Hessian(model, data)
CI, HPSD = CIs(H);

Check the eigenvalues of the hessian, to ensure that it is positive semidefinite

In [ ]:
using LinearAlgebra
eigvals(H)

In [ ]:
xf = collect(Flatten.flatten(model.θ));

Check that the solution is within the confidence intervals

In [ ]:
(xf - CI) .< xgen, (xf + CI) .> xgen

Get the step-by-step values of the optimization and plot them

In [ ]:
trace = hcat(map(x-> x.metadata["x"], output.trace)...);

In [ ]:
using PyPlot

In [ ]:
num_rows, num_cols = 3,3
fig, ax = subplots(num_rows, num_cols, figsize=(9,9))
name = ["σ2_i", "B", "λ", "σ2_a", "σ2_s", "ϕ", "τ_ϕ", "bias", "lapse"]

for i in 1:9
                  
    ax[i].plot(trace[i,:])
    ax[i].plot(xgen[i] * ones(size(trace,2)))
    ax[i].set_title(name[i])
    
    ax[i].errorbar(size(trace, 2), xf[i], yerr=CI[i], fmt="o",
        capsize=6)
end

tight_layout() 
display(gcf())

Compute the LL over the optimization domain, for plotting

In [ ]:
@everywhere ℓℓ(x) = -PulseInputDDM.loglikelihood(x, model, data)

In [ ]:
αs = hcat(map((lb,ub)-> range(lb + eps(), stop=ub, length=30), lb, ub));

In [ ]:
LL_scan = map(i-> map(x-> ℓℓ(vcat(xf[1:i-1], x, xf[i+1:end])), αs[i]), 1:9);

In [ ]:
num_rows, num_cols = 3,3
fig, ax = subplots(num_rows, num_cols, figsize=(9,9))
name = ["σ2_i", "B", "λ", "σ2_a", "σ2_s", "ϕ", "τ_ϕ", "bias", "lapse"]

for i in 1:9
                  
    ax[i].plot(αs[i], LL_scan[i], "x")
    ax[i].set_title(name[i])
    ax[i].plot(xgen[i]*ones(100), range(minimum(LL_scan[i]), stop=maximum(LL_scan[i]), length=100), "k")
    ax[i].plot(xf[i]*ones(100), range(minimum(LL_scan[i]), stop=maximum(LL_scan[i]), length=100),
        "g--")
    ax[i].plot(max((xf[i] - CI[i]), lb[i]) *ones(100), range(minimum(LL_scan[i]), 
            stop=maximum(LL_scan[i]), length=100), "r--")
    ax[i].plot(min((xf[i] + CI[i]), ub[i]) *ones(100), range(minimum(LL_scan[i]), 
            stop=maximum(LL_scan[i]), length=100), "r--")

    
end

tight_layout() 
display(gcf())